# MSII Solution Process

## About:
The Design Project Cadet Handout allows for "a spreadsheet, computer program (e.g., MatLab) or hand calculations to support your analysis," provided that your final submission is "formatted in such a way that the instructor can see the formulas you are using and the final answers." Python is a computational tool of a very similar caliber to MatLab (in the hands of an experienced user), and is thus authorized. A brief in-class discussion with Gen. Thompson confirmed this.

I have, to this end, written a fairly comprehensive software library which is capable of computing, among other things, the percent coverage, budget calculations, orbit positions by time, viability assessments, and many other rudimentary computations of a given constellation. The exact code used for the entire project will be included in the submission, any necessary explanations as to how it works.

Understand that the code in this notebook will be extremely abstracted and surface level because it relies on the software library (entirely written by me) for all the computations that would truly be considered Astro-related.



## MS II Checklist:
### Pre-Checklist Constellation Computations:

#### Import all the necessary code from our custom software library:

In [1]:
# handle all the necessary imports from our custom software library
from MSII_Components import *
from MSII_useful_consts import *
from MSII_Constellation import MSII_Constellation, MSII_Satellite
from MSII_Orbit import MSII_Orbit
from MSII_Constraints import *
from Formula_Sheet import *

#### Create a modifyable template of our base constellation (already has good coverage):

In [2]:
'''
NOTE: We are no longer using the MSI constellation template because we have determined that it is bad. We came up with a new one.
for further details, please see MSII_Base_Orbit_Determination.ipynb, where we did all the computations and reasoning.
'''

# def create_MSI_constellation(payload, adcs, structure):
#     msI_a = GEOSYNCHRONOUS_SEMIMAJOR_AXIS # 42241km from MSI
#     max_height = get_payload_max_height(payload, MIN_RESOLUTION) # how high can we get and still hit our
#     # target resolution (formula derivation done and included on paper) in km

#     # from formula sheet: Ra = a(1 + e)
#     # it follows: e = (Ra / a) - 1 = ((h + Rearth) / a) - 1
#     max_eccentricity = ((max_height + R_EARTH) / msI_a) - 1
#     max_eccentricity = abs(max_eccentricity)
#     max_eccentricity = 0.834

#     return { 
#         "payload": payload,
#         "adcs": adcs,
#         "structure": structure,
#         "sats": [ # we update all the fields that will change with new MSII requirements (just e)
#             {
#                 "a": msI_a,
#                 "e": max_eccentricity,
#                 "i": 50.825, # from MSI
#                 "raan": 340.5, # from MSI
#                 "w": 245, # from MSI
#                 "v": 180 # from MSI
#             }, {
#                 "a": msI_a,
#                 "e": max_eccentricity,
#                 "i": 50.825, # from MSI
#                 "raan": 100.5, # from MSI
#                 "w": 245, # from MSI
#                 "v": 154.5099795 # from MSI
#             }, {
#                 "a": msI_a,
#                 "e": max_eccentricity,
#                 "i": 50.825, # from MSI
#                 "raan": 220.5, # from MSI
#                 "w": 245, # from MSI
#                 "v": 205.4900205 # from MSI
#             }
#         ]
#     }

# take a hardware configuration idea, turn it into a constellation
def create_geosync_constellation(idea_payload=PAYLOADS[0], idea_adcs=ADCS[2], idea_struct=STRUCTURES[2]) -> MSII_Constellation:
    idea = ["Geo-Sync Perigee", h_to_sec(24)]

    altitude_offset = get_payload_max_height(idea_payload, MIN_RESOLUTION) - (MIN_DRAGLESS_ALTITUDE + 1) # +1 to prevent rounding errors
    idea_period = idea[1] # store the target period of the idea
    idea_max_h = get_payload_max_height(idea_payload, MIN_RESOLUTION) # how high can we approach from?
    
    idea_a = period_to_axis(idea_period) # calculate a from period (in seconds)
    idea_i = get_avg_latitude(POINTS_OF_INTEREST) # average out the inclinations of each point
    if (idea_i < 0):
        idea_i += 180 # allow for points south of the equator, should not be a problem
    idea_e = perigee_h_to_e(idea_a, idea_max_h - altitude_offset, False) # what is our max eccentricity to get that height?

    idea_w = 90 # we want to set argument of perigee to 90 to simplify calculations
    if (get_avg_latitude(POINTS_OF_INTEREST) < 0):
        idea_w = 270 # allow for points south of the equator, should not be a problem
    
    idea_sat_coes = []
    for sat_num in range(NUM_SATS):
        time_on_target = (idea_period / NUM_SATS) * sat_num # stagger the times on target evenly
        
        sat = {}
        sat["a"] = idea_a # initialize all the variables that stay the same accross sats
        sat["e"] = idea_e
        sat["w"] = idea_w
        sat["i"] = idea_i
        
        # now for the hard bit, we need to change raan and v so they each hit apogee over
        # the target points at staggered times
        avg_long = get_avg_longitude(POINTS_OF_INTEREST)
        # the raan that would put us over the points in BORG at t=0
        avg_raan_t_0 = avg_long - INTIAL_EARTH_POS - idea_w
        # the raan that would put us over the points in BORG at time on target
        avg_raan_t = avg_raan_t_0 + ((360 / DAY_AS_SECONDS) * time_on_target) # assume 24 hour Earth period
        sat["raan"] = avg_raan_t # set that as our raan
        sat["v"] = calc_v_at_t0_to_hit_perigee_at_t(idea_a, idea_e, time_on_target)
        idea_sat_coes.append(sat) # add the satellite coes to our list
    
    assert(len(idea_sat_coes) == NUM_SATS) # make sure we have the right amount of sats
    
    idea_constellation_dict = {
        "structure": idea_struct, # use the best structure by default so it fits the camera for sure
        "adcs": idea_adcs, # best adcs by default
        "payload": idea_payload, # same with the imager
        "sats": idea_sat_coes
    }
    
    return idea_constellation_dict # wrap it all nicely in a constellation dictionary

#### Generate all possible hardware configurations

In [3]:
constellation_options = []
for payload in PAYLOADS: # find every possible payload-adcs-structure combination
    for adcs in ADCS:
        for struct in STRUCTURES:
            # apply our template from before (which includes modified e computations)
            constellation_object = MSII_Constellation(create_geosync_constellation(payload, adcs, struct))
            constellation_options.append(constellation_object)

# output how many we found
print(f"Found {len(constellation_options)} possible constellations.")

viable_constellation_options = []
for constellation in constellation_options:
    # filter out all the ones that obviously fail (i.e. out of budget, crash into earth, fail volume constraints)
    if (constellation.get_concise_summary().passes_inspection):
        viable_constellation_options.append(constellation)
    # print(constellation.assess_orbit_viability().to_string()

# output how many we found
print(f"Found {len(viable_constellation_options)} viable constellation options")

Found 36 possible constellations.
Found 24 viable constellation options


#### Figure out which one has the best coverage
Note that the algorithm for this is complicated and is included in the algorithm explanation portion of the project.

In [4]:
# we are having slight issues with our coverage computation algorithm, probably something to do with the
# fact that the Earth position and the orbits are out of sync 

best_coverage = 0 # assume zero coverage to start, update later
best_constellation = None

for constellation in viable_constellation_options:
    constellation : MSII_Constellation
    coverage_report = constellation.assess_coverage()
    if (not coverage_report.adcs_cam_compatible):
        # our pointing accuracy is so bad we cannot reliably point at anything
        # therefore, do not count this option's coverage in the contest
        continue

    if (coverage_report.percent_coverage > best_coverage):
        best_coverage = coverage_report.percent_coverage
        best_constellation = constellation
    elif (coverage_report.percent_coverage == best_coverage):
        # if they have the same coverage but one is cheaper, choose that one
        if (best_constellation.assess_budget().cost > constellation.assess_budget().cost):
            best_coverage = coverage_report.percent_coverage
            best_constellation = constellation


print(f"Found constellation with {best_coverage}% coverage")

# output the constellation
print(best_constellation.to_string("#1"))
print(best_constellation.get_comprehensive_summary().to_string("#1"))




Found constellation with 0.06525% coverage
Constellation #1:
	Payload:   Astra Hi-Res Imager
	Structure: Option III
	ADCS:      Option II
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.8347785125143755
		i    : 50.824999999999996
		raan : 290.375
		w    : 90
		v    : 0.0
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.8347785125143755
		i    : 50.824999999999996
		raan : 50.375
		w    : 90
		v    : 146.44029712684926
	ORBIT: 
		a    : 42241.097730143825
		e    : 0.8347785125143755
		i    : 50.824999999999996
		raan : 170.375
		w    : 90
		v    : 169.65931160912885

Constellation #1 Comprehensive Summary: 
	Constellation #1 Concise Summary:
		Passes:       True
		In Budget:    True
		Viable Orbit: True
		Volume Fits:  True
	
	Constellation #1 Orbit Viability: VIABLE
		ORBIT 1
			CATASTROPHIC?      False
			DRAGLESS?          True
			CRASHLESS?         True
			Max Altitude:      71124.92146028765 km
			Min Altitude:      600.9999999999991 km
			Is Geosynchronous: True
		
		ORBIT 2
			CA

## Alright! Now that we have our orbit, we can prove it against the checklist.

#### 7. Finalize your payload choice based on mission requirements and sensor specifications:

In [5]:
import json
print(json.dumps(best_constellation.payload, indent=2)) # output our finalized payload choice

{
  "name": "Astra Hi-Res Imager",
  "min_wl": 3.8e-07,
  "max_wl": 7.5e-07,
  "det_rad": 0.15,
  "fl": 4,
  "aperture": 1.2,
  "hor_pix": 30000,
  "vert_pix": 500,
  "bit_pix": 12,
  "length": 2,
  "width": 1.5,
  "height": 3,
  "tot_mass": 200,
  "tot_pow": 225,
  "cost": 31000000,
  "delivery": 12
}


a. Choose the required resolution for your mission. Details of the target of interest are
included in the Appendix of the Design Project handout. Remember that resolution
defines the distance needed to distinguish objects. Think about what you must be able
to see to positively identify the target(s) associated with your mission.

We require a resolution of 4m because that is the width of the tanks we are looking to spot. 

In [6]:
print(f"Min Resolution: {MIN_RESOLUTION} m") # we stored it as a constant for computations

Min Resolution: 4 m


b. Select a payload from Table 2 and note the range of wavelengths.

We selected the Astra High-Res Imager, because it has the lowest max wavelength. This increases our resolution at higher altitudes.

In [7]:
print(f"Name:           {best_constellation.payload["name"]}")
print(f"Max Wavelength: {best_constellation.payload["max_wl"]} m")
print(f"Min Wavelength: {best_constellation.payload["min_wl"]} m")

Name:           Astra Hi-Res Imager
Max Wavelength: 7.5e-07 m
Min Wavelength: 3.8e-07 m


c. Calculate required orbital altitudes based on your selected mission resolution and the
range of wavelengths. Select a specific mission orbit altitude using these constraints.
You must meet the resolution requirement at both the min and max wavelength and at
the min and max mission altitude (if using an elliptical orbit).

We designed our orbit such that it is always at perigee when it goes over the points (without fail), and such that it is always at the mission altitude when it reaches perigee. See MSII_Base_Orbit_Determination.ipynb for details on how this was ensured. See Formula_Sheet.py for the exact formula used to compute mission altitude. The requirement to meet the resolution requirement at apogee too is clearly for orbits that are not properly synchronized with the 24 hour period of the Earth. Ours is, meaning that -- without fail -- our apogee is somewhere over the Pacific Ocean. There is no need to have 4m resolution images of the Pacific Ocean at this point in time. Details for why an orbit entirely within mission altitude is a bad idea are also included in MSII_Base_Orbit_Determination.ipynb.

In [8]:
# this uses the max wavelength. Because we have to hit our target resolution at both our min and max wavelength,
# and because our resolution at max wavelength is always lower than at min wavelength, our maximum altitude is 
# really only going to be dictated by max wavelength. The minimum wavelength is completely irrelevant to our orbit
# altitudes
print(f"Required Orbital Altitude: {get_payload_max_height(best_constellation.payload, MIN_RESOLUTION)} km")

# According to class, negligible drag starts at 600km out. This is interesting. Really, there is no reason to 
# go any higher than this at perigee, because by bringing perigee down to 600km, we can increase the section of 
# our orbit where we can see the points. Because of this:
mission_alt = 600
print(f"Chosen Mission Orbital Altitude: {mission_alt} km")

Required Orbital Altitude: 1074.9798441279227 km
Chosen Mission Orbital Altitude: 600 km


d. Calculate field of view and swath width for your chosen mission altitude and payload.

Will do! See Formula_Sheet.py for the exact formulas used for this and many other things.

In [9]:
# call the corresponding functions from our formula sheet
print(f"Swath Width at Mission Altitude: {calc_swath_width(best_constellation.payload, mission_alt)} km")
print(f"Selected Payload FOV: {calc_payload_fov(best_constellation.payload)} deg")

Swath Width at Mission Altitude: 45.0 km
Selected Payload FOV: 4.295170856597006 deg


#### 8. Update the Classical Orbital Elements for each satellite in the constellation.

a. Calculate the eccentricity and semi-major axis for each orbit. Decide whether the
mission orbit for each satellite should be circular or elliptical. The eccentricities of the
orbits can be the same, or they may be different based on the choices made in
paragraph 1 above.

We already did this when we were designing our orbit template in MSII_Base_Orbit_Determination.ipynb, but we will reproduce the process here for clarity. All formulas for this, you are reminded, can be found in the Formula_Sheet.py file. We did elliptical so we can stay geosynchronous while still flying over the points at mission altitude.

In [10]:
semimajor_axis = period_to_axis(h_to_sec(24)) # convert our 24 hour period to a
eccentricity = perigee_h_to_e(semimajor_axis, 600, False) # what is our max eccentricity to get that height at perigee?

# we use these same values for each orbit
print(f"a: {semimajor_axis} km")
print(f"e: {eccentricity}")

a: 42241.097730143825 km
e: 0.8348021861415712


b. Select the inclination for each mission orbit. Ensure the constellation provides
suitable coverage of the assigned monitoring area.

This is easy, just take the average latitude of all the points. Again, done elsewhere already, reproduced below for clarity.

In [11]:
inclination = get_avg_latitude(POINTS_OF_INTEREST)
print(f"i: {inclination} deg")

i: 50.824999999999996 deg


c. Choose a value for Right Ascension of the Ascending Node (or Special Case COE)
for each orbit. This should be based on the overall constellation design and
consideration for how often the target area may be in-view of your satellites. Note: You
do not need to do any calculations related to re-visit time.

This has already been covered exhaustively in MSII_Base_Orbit_Determination.ipynb, which is attached with this submission. Because I took an extremely mathematical approach, reproducing it here is impractical. The entire orbit design process is already documented there. Here is my selected values:

In [12]:
for sat in best_constellation.sats:
    print(f"RAAN: {sat.orbit.raan} deg")

RAAN: 290.375 deg
RAAN: 50.375 deg
RAAN: 170.375 deg


d. Choose a value for Argument of Perigee (or Special Case COE) as appropriate for
each orbit.

Again, same deal.

In [13]:
for sat in best_constellation.sats:
    print(f"Argument of Perigee: {sat.orbit.w} deg")

Argument of Perigee: 90 deg
Argument of Perigee: 90 deg
Argument of Perigee: 90 deg


e. Choose an initial value for True Anomaly (or Special Case COE) as appropriate for
each satellite.

The math for this one in particular, is quite insane. PLEASE consult the MSII_Base_Orbit_Determination.ipynb, it contains the vast majority of the computational heavy lifting. Also, see the "calc_v_at_t0_to_hit_perigee_at_t" function defined in Formula_Sheet.py, it has most of the actual formulas for this, along with detailed explanations of how it all works. The answers derived from those computations are below.

In [14]:
for sat in best_constellation.sats:
    print(f"True Anomaly: {sat.orbit.v} deg")

True Anomaly: 0.0 deg
True Anomaly: 146.44029712684926 deg
True Anomaly: 169.65931160912885 deg


#### 9. Select a structure for each satellite from Table 4. Your payload must fit inside your structure’s internal payload volume.

We kind of picked all three components together as a package deal. We tested each possible combination of adcs, payload, and structure, and picked the one that was the best. Here is the breakdown:

In [15]:
print("Chosen Structure: " + json.dumps(best_constellation.structure, indent=2))
print("\n")
# see MSII_Constellation.py and MSII_Constraints Volume Summary for this function definition
# it is pretty simple / straightforward, though
print(best_constellation.assess_volume().to_string("#1")) 

Chosen Structure: {
  "name": "Option III",
  "internal volume": [
    2,
    2,
    3
  ],
  "external side length": 3.5,
  "spring constant": 34000000.0,
  "mass": 215,
  "cost": 125000,
  "delivery": 5
}


Constellation #1 Volume Summary:
	Payload Fits:     True
	Available Volume: 12 m^3
	Used Volume:      9.0 m^3
	Remaining Volume: 3.0 m^3



#### 10. Select an Attitude Determination and Control Subsystem (ADCS) from Table 5 and calculate the worst-case ground pointing accuracy (GPA) using the angular pointing accuracy of the ADCS option selected.

Done, done, and done! Again, we picked a combination of adcs, payload, and structure. We picked this by testing every possible combination. That work is shown at the top of this file.

In [16]:
print("Chosen ADCS: " + json.dumps(best_constellation.adcs, indent=2))
print("\n")

print(f"Ground Pointing Accuracy (worst case): {get_ground_pointing_accuracy(best_constellation.adcs, mission_alt)} km")

Chosen ADCS: {
  "name": "Option II",
  "pointing accuracy": 0.035,
  "power": 40,
  "mass": 28,
  "cost": 375000,
  "delivery": 6
}


Ground Pointing Accuracy (worst case): 21.000000000000004 km
